<a href="https://colab.research.google.com/github/hanfengzhai/PIDOC/blob/main/vanderPol/vanderPol_Fig4_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow==2.8.0
!pip install tensorflow==1.15.0
import tensorflow as tf
# import deepxde as dde
import numpy as np
from scipy import linspace
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import scipy.io
import time
import timeit
# import tensorflow_probability as tfp

# tf.disable_v2_behavior()
print(tf.__version__)

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 3.8 MB 49.0 MB/s 
     |████████████████████████████████| 503 kB 60.8 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=b192539ce65e5c1b88a6b79fdc8f61c12d290ab0836832e4ea09eb3ece3a5c4a
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32

In [2]:
class DeepvdP:
    # Initialize the class
    def __init__(self, x, t, layers):

        self.lb = t.min(0)
        self.ub = t.max(0)
        
        self.x = x
        self.t = t
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])

        self.x_control = 2 * tf.math.sin(self.t_tf)#5.00 *
        self.x_dd_control = - 2 * tf.math.sin(self.t_tf) #5.00 *

        self.x_pred, self.x_dd_pred, self.ICs = self.vdP(self.t_tf)

        self.x_res = self.x_control - self.x_pred
        self.x_dd_res = self.x_dd_control - self.x_dd_pred

        self.control = self.x_res + self.x_dd_res

        self.loss = 1 * tf.reduce_sum(tf.square(self.control)) + tf.reduce_sum(tf.square(self.ICs)) + \
        tf.reduce_sum(tf.square( self.x_tf - (self.x_pred) )) #+ \* 2 | * 2 / 5
                    

        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 200000,
                                                                           'maxfun': 200000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps}) 


        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
# ==============================================================================================
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
   
    def neural_net(self, t, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(t - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
# ==============================================================================================
    def vdP(self, t):
      x = self.neural_net(tf.concat([t],1), self.weights, self.biases)
      # dx_t = dde.grad.jacobian(x, t, i=0)
      # dx_tt = dde.grad.hessian(x, t, i=0)
      dx_t = tf.compat.v1.gradients(x, t)
      dx_tt = tf.compat.v1.gradients(dx_t, t)
      # x_desire = tf.math.sin(t) #5 * 
      # x_dot_desire = tf.math.cos(t) #5 * 
      # x_ddot_desire = - tf.math.sin(t) #-5 * 
      # control = (x_desire - x) + (x_ddot_desire - dx_tt)
      ICs = x[0] - 1# Initial condition: 1, 5, 10
      return x, dx_tt, ICs
# ==============================================================================================
    def callback(self, loss): #, betta
        print('%.3e' % (loss)) #, betta B: %.5f
        return loss
      
    def train(self, nIter): 

        tf_dict = {self.x_tf: self.x, self.t_tf: self.t}

        # var_loss = tf.Variable(tf_dict)
        # loss = lambda: (var_loss ** 2)/2.0 

        self.sess.run(self.train_op_Adam, feed_dict = tf_dict)

        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)

        # self.optimizer.minimize(self.loss)
        # self.optimizer.minimize(self.loss, global_step=None, var_list=var_loss,
    # aggregation_method=None, colocate_gradients_with_ops=False, name=None,
    # grad_loss=None)

        # (self.sess, feed_dict = tf_dict, fetches = [self.loss], loss_callback = self.callback)
            
# ==============================================================================================    
    def predict(self, t_star):
        
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t}  
        x_star = self.sess.run(self.x_pred, tf_dict)
        
        return x_star

In [3]:
if __name__ == "__main__": 
    
    layers = [1, 30, 30, 30, 30, 30, 30, 1]#30, 30, 30, 30, \
                #  30, 30, 30, 30, 30, 30, 30, 30, 30, 30, \
                #  30, 30, 30, 30, 30, 30, 30, 30, 30, 30, \
                #  30, 30, 30, 30, 30, 30, 30, 30, 30, 30, \
                #  30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 
    # Set NN Structure
    
    data = scipy.io.loadmat('train.mat')
    t_obtain = data['t'] # 
    X_obtain = data['x'] # 
    t_star = t_obtain[0:3000]
    # t_star = t_star 
    X_star = X_obtain[0:3000]
    # print(sol.t)
    # print(sol.y[0])
    N = X_star.shape[0]
    T = t_star.shape[0]
    # print(X_star)
    # print(t_star)
    # Rearrange Data 
    XX = np.tile(X_star, (1,T)) # [0:3000]
    TT = np.tile(t_star, (1,N)).T # 
    
    x = XX.flatten()[:,None] #
    t = TT.flatten()[:,None] #

    # Training Data    
    x_train = X_star #x[:,:] # [idx,:]
    # noise = 0.00
    # x_train = x_train * (1 + noise*np.random.standard_normal(3000))
    t_train = t_star #t[:,:]

    # Training
    t_tic = time.time()
    model = DeepvdP(x_train, t_train, layers)
    model.train(200000)
    # cpu_time = timeit.default_timer()
    # Prediction
    x_pred = model.predict(t_star)
    # loss_hist = model.callback()
    elapsed_toc = time.time() - t_tic

    np.savetxt("prediction.txt", np.hstack(x_pred))
    print(elapsed_toc)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

5.952e+03
7.755e+03
5.951e+03
5.949e+03
5.948e+03
5.948e+03
5.948e+03
5.948e+03
5.947e+03
5.946e+03
5.942e+03
5.943e+03
5.941e+03
5.941e+03
5.940e+03
5.936e+03
5.935e+03
5.935e+03
5.933e+03
5.931e+03
5.929e+03
5.929e+03
5.929e+03
5.928e+03
5.927e+03
5.926e+03
5.945e+03
5.925e+03
5.925e+03
5.925e+03
5.924e+03
5.938e+03
5.923e+03
5.927e+03
5.922e+03
5.922e+03
5.922e+03
5.921e+03
5.920e+03
5.919e+03
5.918e+03
5.927e+03
5.917e+03
5.916e+03
5.916e+03
5.915e+03
5.914e+03
5.914e+03
7.567e+03
5.914e+03
5.914e+03
6.126e+03
5.913e+03
6.863e+03
5.91